In [1]:
from transformers import WhisperProcessor
from transformers import WhisperForConditionalGeneration
from transformers import Seq2SeqTrainer
from transformers import Seq2SeqTrainingArguments
import librosa
import torch
import numpy as np
import pandas as pd
import torch
import torch.optim as optim
from transformers import pipeline
import gradio as gr
from evaluate.evaluator import text2text_generation
from transformers import pipeline
from evaluate.evaluator import text2text_generation
from transformers import pipeline
import gradio as gr
import librosa
import pandas as pd
import numpy as np
import pickle
from datasets import Dataset, DatasetDict
import transformers
import torch
import evaluate
import sys 

from dataclasses import dataclass
from typing import Any, Dict, List, Union

# Load the list from the file
with open("/home/yadinb/dharelg/yadinb/alona_tag_data_POC.pkl", 'rb') as f:
    df = pd.DataFrame(pickle.load(f))


In [6]:
df.iloc[0].audio

{'path': '/home/eranbe/dharelg/datasets/whisper_dataset/prototype_dataset/TAL_4prototype_tmpfiles_4creating_dataset/wav/0.wav',
 'array': array([-0.07229415, -0.10425258, -0.08992703, ..., -0.01851533,
        -0.03886626,  0.        ]),
 'sampling_rate': 16000}

In [ ]:
%%writefile training_script_working.py
from transformers import WhisperProcessor
from transformers import WhisperForConditionalGeneration
from transformers import Seq2SeqTrainer
from transformers import Seq2SeqTrainingArguments
import librosa
import torch
import numpy as np
import pandas as pd
import torch
import torch.optim as optim
from transformers import pipeline
import gradio as gr
from evaluate.evaluator import text2text_generation
from transformers import pipeline
from evaluate.evaluator import text2text_generation
from transformers import pipeline
import gradio as gr
import librosa
import pandas as pd
import numpy as np
import pickle
from datasets import Dataset, DatasetDict
import transformers
import torch
import evaluate
import sys 

from dataclasses import dataclass
from typing import Any, Dict, List, Union

# Load the list from the file
with open("/home/yadinb/dharelg/yadinb/alona_tag_data_POC.pkl", 'rb') as f:
    df = pd.DataFrame(pickle.load(f))

for index, row in df.iterrows():
    row['audio']['array'] = np.array(row['audio']['array'], dtype=np.float32)

    
model_arch = "small"
print("model_arch:",model_arch)
output_dir = f"/home/moshebr/dharelg/moshe/recreation_script_again_w_{model_arch}"

seed = 42  # Or any other number you like

train_df = df.iloc[:int(0.95*len(df))]
train_df = train_df.sample(frac=1, random_state=seed).reset_index(drop=True)  # shuffle and reset index
train_dataset = Dataset.from_pandas(train_df)



# Create the Dataset objects
#train_dataset = Dataset.from_pandas(df.iloc[:int(0.95*len(df))])




test_dataset = Dataset.from_pandas(df.iloc[int(0.95*len(df)):])

# Create the DatasetDict object
dataset_dict = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

TEST_DF = df.iloc[int(0.95*len(df)):]
feature_extractor = transformers.WhisperFeatureExtractor.from_pretrained(f"openai/whisper-{model_arch}")

tokenizer = transformers.WhisperTokenizer.from_pretrained(f"openai/whisper-{model_arch}", language="english", task="transcribe")

processor = transformers.WhisperProcessor.from_pretrained(f"openai/whisper-{model_arch}", language="english", task="transcribe")
#need to trancate sequence ,model works only with 1024 token.
#@@@@@@@@@@
#@@@@@@@@@@
#$$$$$$$$$$$$$$$$$
#MAX_LENGTH = 1024
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    # truncate target text to a maximum length
    #sentence = batch["sentence"][:MAX_LENGTH]
    sentence = batch["sentence"]


    """

    # split the string into list of words
    words = sentence.split()

    # define the markers
    markers = ['TRUE1TRUE', 'FALSE1TRUE', 'TRUE2TRUE', 'FALSE2TRUE', 'TRUE3TRUE', 'FALSE3TRUE', 'TRUE1FALSE', 'FALSE1FALSE', 'TRUE2FALSE', 'FALSE2FALSE', 'TRUE3FALSE', 'FALSE3FALSE']

    # process the words
    output_words = [word.lower() if word not in markers else word for word in words]

    # join the words back into a string
    sentence = ' '.join(output_words)

    """





    # compute input length of audio sample in seconds
    batch["input_length"] = len(audio["array"]) / audio["sampling_rate"]
    # encode target text to label ids
    labels_ids = tokenizer(sentence).input_ids
    # truncate labels to a maximum length
    #labels_ids = labels_ids[:MAX_LENGTH]
    batch["labels"] = labels_ids
    return batch
dataset = dataset_dict.map(prepare_dataset, remove_columns=dataset_dict.column_names["train"], num_proc=8)
max_input_length = 30
min_input_length = 0


def is_audio_in_length_range(length):
    return length > min_input_length and length <= max_input_length
vectorized_datasets = dataset.filter(
    is_audio_in_length_range, num_proc=1, input_columns=["input_length"]
)

model = transformers.WhisperForConditionalGeneration.from_pretrained(f"openai/whisper-{model_arch}")
model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language="english", task="transcribe")
model.config.suppress_tokens = []
max_label_length = model.config.max_length
def is_labels_in_length_range(labels):
    return len(labels) < max_label_length
common_voice = vectorized_datasets.filter(
    is_labels_in_length_range, num_proc=1, input_columns=["labels"]
)

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

metric = evaluate.load("wer")
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

training_args = transformers.Seq2SeqTrainingArguments(
    output_dir =output_dir,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=1000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=1,
    predict_with_generate=True,
    generation_max_length=448,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["none"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)

trainer = transformers.Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)
processor.save_pretrained(training_args.output_dir)
trainer.train()
